## Imports

* needs [PhyloProPy](https://github.com/felixlangschied/PhyloProPy) package

In [1]:
import pandas as pd
from PhyloProPy.PhyloProfile import PhyloProfile
from ete3 import NCBITaxa
import numpy as np
from collections import defaultdict, Counter
from tqdm import tqdm
import ast
from pathlib import Path

## Files

In [2]:
project_dir = Path('.').resolve().parents[2]
seedgeneexcel = f'{project_dir}/shared_data/curated_seedgene_info.xlsx'
seedgenetsv = f'{project_dir}/shared_data/curated_seedgene_info.tsv'
luca_profile = '../data/eukaryots_cell_wall.phyloprofile'
phyloprofile = '../data/eukaryots_cell_wall.phyloprofile'
milestonedir = f'{project_dir}/milestones/representative_seedgenes'

## Variables

In [3]:
max_counter_to_merge = 2

## Filter phyloprofile to eukaryots

### Find all ncbi taxonomy IDs

In [4]:
taxids = set()
with open(luca_profile) as fh:
    header = next(fh)
    for line in fh:
        taxid = int(line.split()[1].replace('ncbi', ''))
        taxids.add(taxid)

### Check which IDs are eukaryotic

In [5]:
from ete3 import NCBITaxa


def find_lineage_taxa(taxids, lineage):
    ncbi = NCBITaxa()
    lineage_id = list(ncbi.get_name_translator([lineage]).values())[0][0]

    in_lineage = set()
    for taxid in taxids:
        lineage = ncbi.get_lineage(taxid)
        if lineage_id in lineage:
            in_lineage.add(f'ncbi{taxid}')
    return in_lineage

eukaryote_ids = find_lineage_taxa(taxids, "Eukaryota")

### Filter profile

In [6]:
def filter_profile(path, outpath, taxids):
    with open(path) as fh:
        header = next(fh)
        profile_col = [header]
        for line in fh:
            if line.split()[1] in taxids:
                profile_col.append(line)
    
    with open(outpath, 'w') as of:
        for line in profile_col:
            of.write(line)


filter_profile(luca_profile, phyloprofile, eukaryote_ids)

## Load seedgene information

In [7]:
seeddf = pd.read_excel(seedgeneexcel)

# save as tsv for later use
seeddf.to_csv(seedgenetsv, index=False, sep='\t')

# check for duplicates
uniprot_count = Counter(seeddf['#_Prot_ID'])
for key, count in uniprot_count.items():
    assert count == 1, f'Duplicated uniprot ID in seedgene table: {key}'

## Map seedgenes to seed taxid 

* the phyloprofile needs to be ordered according to the taxid that served as seed for the fDOG search

In [8]:

taxid2seedgenes = {}
taxid2groups = {}
for taxid in seeddf['#_Seed_taxon_ID'].unique():
    tdf = seeddf[seeddf['#_Seed_taxon_ID'] == taxid]
    
    taxid2seedgenes[taxid] = list(tdf['#_Seed_ID_phyloprofile'].unique())
    taxid2groups[taxid] = list(tdf['#_GH_group'].unique())

print(taxid2seedgenes)
print(taxid2groups)

{1138384: ['GH124_ALX08796.1', 'GH26_ALX07411.1', 'GH26_ALX09217.1', 'GH26_ALX09255.1', 'GH39_ALX09806.1', 'GH44_ALX08608.1', 'GH48_ALX09176.1', 'GH48_ALX09761.1', 'GH8_ALX08975.1'], 6500: ['GH5_10_XP_012938873.1', 'CE13_XP_005111140.1'], 469383: ['GH116_ADB52396.1'], 660027: ['GH64_QKD55153.1', 'GH64_QKD57507.2', 'GH64_QKD57567.2', 'GH64_QKD58554.1', 'GH81_QKD56309.1'], 456999: ['GH1_QRW24047.1', 'GH1_QRW26028.1', 'GH10_QRW19256.1', 'GH10_QRW19862.1', 'GH10_QRW19865.1', 'GH10_QRW19908.1', 'GH10_QRW19954.1', 'GH10_QRW22205.1', 'GH10_QRW22206.1', 'GH10_QRW22755.1', 'GH10_QRW25572.1', 'GH12_QRW15727.1', 'GH12_QRW15728.1', 'GH12_QRW23323.1', 'GH12_QRW23340.1', 'GH128_QRW16590.1', 'GH128_QRW18084.1', 'GH128_QRW18428.1', 'GH128_QRW18429.1', 'GH128_QRW19219.1', 'GH128_QRW19736.1', 'GH128_QRW20133.1', 'GH128_QRW23777.1', 'GH131_QRW16484.1', 'GH131_QRW19984.1', 'GH131_QRW22550.1', 'GH16_QRW16832.1', 'GH16_QRW22852.1', 'GH16_QRW22853.1', 'GH16_1_QRW16739.1', 'GH16_1_QRW19024.1', 'GH16_1_QRW1902

## Load PhyloProfile

In [9]:
pp = PhyloProfile(phyloprofile, from_custom=False, style='orthoid')
ncbi = pp.ncbi

# check that all expected seedgenes are in the profile
for reftaxid, seedgenes in taxid2seedgenes.items():
    for seedgene in seedgenes:
        assert seedgene in pp.matrix.index, f'{seedgene} was not found in phyloprofile'
pp.display()

2024-09-11 11:43:39 - INFO - Reading NCBI Taxonomy
2024-09-11 11:43:39 - INFO - Initializing PhyloProfile matrix
2024-09-11 11:43:39 - INFO - Loading PhyloProfile matrix


,ncbi5866,ncbi41047,ncbi31271,ncbi208452,ncbi119953,ncbi148594,ncbi331117,ncbi419612,ncbi51031,ncbi3641,...,ncbi9606,ncbi185431,ncbi5480,ncbi8018,ncbi881290,ncbi8032,ncbi54757,ncbi6565,ncbi48883,ncbi944170
GH28_QRW21530.1,0,[GH28_QRW21530.1|ASPTH@41047@002237265_1|XP_02...,0,0,0,0,[GH28_QRW21530.1|ASPFI@331117@000149645_2|XP_0...,0,0,[GH28_QRW21530.1|THECA@3641@000208745_1|XP_017...,...,0,0,0,0,0,0,0,0,0,0
GH5_7_QRW26055.1,0,0,0,0,[GH5_7_QRW26055.1|ALTAT@119953@907166805_1|XP_...,0,0,0,0,[GH5_7_QRW26055.1|THECA@3641@000208745_1|XP_00...,...,0,0,0,0,0,0,0,0,0,0
GH18_XP_013955954.1,0,[GH18_XP_013955954.1|ASPTH@41047@002237265_1|X...,0,0,[GH18_XP_013955954.1|ALTAT@119953@907166805_1|...,[GH18_XP_013955954.1|FALNA@148594@017639655_2|...,0,[GH18_XP_013955954.1|CAMFE@419612@009834535_1|...,[GH18_XP_013955954.1|NECAM@51031@000507365_1|X...,0,...,0,0,0,[GH18_XP_013955954.1|ONCKE@8018@012931545_1|XP...,0,[GH18_XP_013955954.1|SALTR@8032@901001165_1|XP...,0,[GH18_XP_013955954.1|CRAVI@6565@002022765_2|XP...,[GH18_XP_013955954.1|GEOFO@48883@000277835_1|X...,[GH18_XP_013955954.1|BLASP@944170@000743755_1|...
PL1_4_QRW23631.1,0,[PL1_4_QRW23631.1|ASPTH@41047@002237265_1|XP_0...,0,0,[PL1_4_QRW23631.1|ALTAT@119953@907166805_1|XP_...,0,[PL1_4_QRW23631.1|ASPFI@331117@000149645_2|XP_...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GH5_7_QRW21487.1,0,[GH5_7_QRW21487.1|ASPTH@41047@002237265_1|XP_0...,0,0,[GH5_7_QRW21487.1|ALTAT@119953@907166805_1|XP_...,0,[GH5_7_QRW21487.1|ASPFI@331117@000149645_2|XP_...,0,0,[GH5_7_QRW21487.1|THECA@3641@000208745_1|XP_00...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GH67_EAA66353.1,0,[GH67_EAA66353.1|ASPTH@41047@002237265_1|XP_02...,0,0,[GH67_EAA66353.1|ALTAT@119953@907166805_1|XP_0...,0,[GH67_EAA66353.1|ASPFI@331117@000149645_2|XP_0...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GH18_UKZ53359.1,0,0,0,0,0,0,0,[GH18_UKZ53359.1|CAMFE@419612@009834535_1|XP_0...,[GH18_UKZ53359.1|NECAM@51031@000507365_1|XP_01...,[GH18_UKZ53359.1|THECA@3641@000208745_1|XP_017...,...,[GH18_UKZ53359.1|HOMSA@9606@000001405_39|NP_00...,0,0,0,0,0,0,0,0,[GH18_UKZ53359.1|BLASP@944170@000743755_1|XP_0...
CBM14_EAA66598.1,0,0,0,0,0,0,[CBM14_EAA66598.1|ASPFI@331117@000149645_2|XP_...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GH11_ABF50866.1,0,[GH11_ABF50866.1|ASPTH@41047@002237265_1|XP_02...,0,0,[GH11_ABF50866.1|ALTAT@119953@907166805_1|XP_0...,0,[GH11_ABF50866.1|ASPFI@331117@000149645_2|XP_0...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Find Fungi taxonomy IDs

* We need those to check if the gene duplication occured in Fungi

In [10]:
fungi_ids = []
for taxid in pp.matrix.columns:
    
    lineage = ncbi.get_lineage(int(taxid.replace('ncbi', '')))
    namedict = ncbi.get_taxid_translator(lineage)

    names = [namedict[taxid] for taxid in lineage]
    if 'Fungi' in names:
        fungi_ids.append(taxid)
print('done')

done


## Define Functions

In [11]:
def compare_lineages(df, max_counter_to_merge):
    # display(df)
    merge_counter = 0
    merge_on_taxid = ''
    for taxid, series in df.items():
        if '0' in series.values:
            continue
        set1 = {geneid.split('|')[2] for geneid in ast.literal_eval(series[0])}
        set2 = {geneid.split('|')[2] for geneid in ast.literal_eval(series[1])}

        if len(set1.intersection(set2)) > 0:
            merge_counter += 1
            merge_on_taxid = taxid
        else:
            merge_counter = 0
            #merge_on_taxid = ''
        if merge_counter == max_counter_to_merge:
            return merge_on_taxid  
  

def pairwise_lineage_comparison(
    groupdf, group, ncbi, fungi_ids, group_metazoa, reftaxid, max_counter_to_merge, maximize
):
    mergedf_taxid = pd.DataFrame(columns=groupdf.index, index=groupdf.index)
    mergedf_lineagerank = pd.DataFrame(columns=groupdf.index, index=groupdf.index)
    mergedf_name = pd.DataFrame(columns=groupdf.index, index=groupdf.index)
    for i in range(groupdf.index.size):
        for j in range(groupdf.index.size):
            if i != j:
                pairwise_compare = groupdf.iloc[[i,j],:]
                merge_on_taxid = compare_lineages(pairwise_compare, max_counter_to_merge)

                if not merge_on_taxid:
                    mergedf_taxid.iloc[i,j] = 'NO MERGE'
                    mergedf_name.iloc[i,j] = 'NO MERGE'
                    mergedf_lineagerank.iloc[i,j] = 'NO MERGE'
                else:
                    # convert taxid to names
                    namedict = ncbi.get_taxid_translator([int(merge_on_taxid.replace('ncbi', ''))])
                    # how deep in the profile (taxonomic tree) did the merge event happen?
                    lineagerank = list(groupdf.columns).index(merge_on_taxid)
                    
                    #save results
                    mergedf_taxid.iloc[i,j] = merge_on_taxid
                    mergedf_name.iloc[i,j] = list(namedict.values())[0]
                    mergedf_lineagerank.iloc[i,j] = lineagerank

    # count the number of unique fDOG hits of each paralog
    gene2num_unique_hits = {}

    for geneid, row in group_metazoa.iterrows():
        unique_hits = set()
        for gene_list in row:
            if gene_list == 0:
                continue
            for gene in gene_list:
                unique_hits.add(gene)
        gene2num_unique_hits[geneid] = len(unique_hits)

    unique_hits = pd.DataFrame.from_dict(
        gene2num_unique_hits, orient='index', columns=['count']
    ).sort_values(by='count', ascending=False)
    

    representatives = set()
    merged = set()
    for geneid in unique_hits.index.values:
        if not representatives:  # always add hit with most unique hits
            representatives.add(geneid)
            merged.add(geneid)
            continue
        merge_ids = set()
        for representative in list(representatives):
            if representative == geneid:
                continue
            merge_ids.add(mergedf_taxid.loc[representative, geneid])
        # does the gene merge with any of the representatives in fungi?
        if len(merge_ids.intersection(set(fungi_ids))) == 0:  
            representatives.add(geneid)
        else:
            merged.add(geneid)
            

    print(f'{group}: Added {len(representatives)} representatives out of {groupdf.index.size} paralogs')
    return list(representatives)
        
    

def get_representatives(groups, xpdf, metazoadf, reftaxid, max_counter_to_merge, maximize):
    representatives = []
    for group in groups:
        groupdf = xpdf[xpdf.index.str.startswith(f'{group}_')]
        
        if groupdf.index.size == 1:
            print(f'{group}: Only one member')
            representatives.append(groupdf.index[0])
        elif groupdf.index.size == 0:
            print(f'{group} not found in profile')
            continue
        else:
            group_metazoa = metazoadf[metazoadf.index.str.startswith(f'{group}_')]
            representatives.extend(
                pairwise_lineage_comparison(groupdf, group, ncbi, 
                                            set(fungi_ids), group_metazoa, 
                                            reftaxid, max_counter_to_merge, maximize
                                           )
        )
    return representatives


# Perform filtering

* Step one: find taxonomic node where gene duplication occured
* step two:
    * Rank paralogs by the number of unique gene_ids of metazoan fDOG hits
    * Add first rank as representative
    * For each other paralog: did it emerge from a fungal gene duplication compared to any representative?
      * If not, add as representative

In [20]:

full_metazoadf = pp.lineage_slice('Metazoa')
representatives = []
for reftaxid, seedgenes in taxid2seedgenes.items():
    #print(seedgenes)
    if f'ncbi{reftaxid}' not in pp.matrix.columns:
        representatives.extend(seedgenes)  # seedgenes not in the filtered profile come from bacteria for which no representatives need to be selected
        continue
    pp.set_reference(reference=reftaxid)
    df = pp.slice(genes=seedgenes)
    xpdf = df.applymap(lambda x: x.split('|')[2] if isinstance(x, str) else x).astype(str)

    metazoadf = full_metazoadf.loc[seedgenes,:]

    groups = taxid2groups[reftaxid]
    representatives.extend(get_representatives(groups, xpdf, metazoadf, reftaxid, max_counter_to_merge, 'Metazoa'))

# remove duplicates
representatives = set(representatives)


2024-09-11 13:09:22 - INFO - Reordering matrix according to "6500"
2024-09-11 13:09:23 - INFO - Reordering matrix according to "660027"


GH5_10: Only one member
CE13: Only one member


2024-09-11 13:09:24 - INFO - Reordering matrix according to "456999"


GH64: Added 2 representatives out of 4 paralogs
GH81: Only one member
GH1: Added 1 representatives out of 2 paralogs
GH10: Added 2 representatives out of 9 paralogs
GH12: Added 2 representatives out of 4 paralogs
GH128: Added 1 representatives out of 8 paralogs
GH131: Added 2 representatives out of 3 paralogs
GH16: Added 6 representatives out of 25 paralogs
GH16_1: Added 3 representatives out of 12 paralogs
GH16_19: Only one member
GH16_2: Added 1 representatives out of 9 paralogs
GH17: Added 2 representatives out of 2 paralogs
GH2: Added 1 representatives out of 2 paralogs
GH3: Added 2 representatives out of 12 paralogs
GH30: Added 2 representatives out of 2 paralogs
GH30_3: Only one member
GH45_1: Added 1 representatives out of 2 paralogs
GH45_4: Only one member
GH5_12: Added 2 representatives out of 2 paralogs
GH5_15: Added 1 representatives out of 5 paralogs
GH5_22: Added 1 representatives out of 2 paralogs
GH5_31: Only one member
GH5_4: Added 1 representatives out of 2 paralogs
GH

2024-09-11 13:11:39 - INFO - Reordering matrix according to "227321"


...

GH19: Added 1 representatives out of 2 paralogs
GH75: Added 1 representatives out of 5 paralogs
GH18: Added 7 representatives out of 34 paralogs
CBM18: Added 3 representatives out of 3 paralogs


2024-09-11 13:12:56 - INFO - Reordering matrix according to "688394"


CBM50: Added 3 representatives out of 10 paralogs


2024-09-11 13:12:57 - INFO - Reordering matrix according to "370354"


GH46: Added 1 representatives out of 2 paralogs
CBM55: Added 2 representatives out of 2 paralogs


## Manual Curation

* Note: Only cellulases and hemi-cellulases were manually curated by Ingo (for the fDOG manuscript)

In [23]:
to_remove = [
        'GH16_1_QRW16739.1',
        'GH16_1_QRW21843.1',
        'GH16_2_QRW24674.1',
        'GH124_ALX08796.1', 
        'GH43_QRW15720.1',
        'PL1_QRW18745.1',
]
for representative in to_remove:
    if representative in representatives:
        representatives.remove(representative)

print(len(set(representatives)))

105


## Check final list of representatives

In [24]:

repdf = seeddf[seeddf['#_Seed_ID_phyloprofile'].isin(representatives)]
display(repdf)

print(Counter(repdf['#_Enzyme']))

,#_No,#_Seed_ID_phyloprofile,#_GH_group,#_Prot_ID,#_Seed_taxon,#_Seed_taxon_ID,#_refspec_fdog,#_Enzyme,#_Description
1,2,GH26_ALX07411.1,GH26,ALX07411.1,Acetivibrio thermocellus AD2,1138384,ACETH@1138384@000255615_3,Cellulase,"Mannan endo-1,4-beta-mannosidase, Cellulase"
2,3,GH26_ALX09217.1,GH26,ALX09217.1,Acetivibrio thermocellus AD2,1138384,ACETH@1138384@000255615_3,Cellulase,"Mannan endo-1,4-beta-mannosidase, Cellulase"
3,4,GH26_ALX09255.1,GH26,ALX09255.1,Acetivibrio thermocellus AD2,1138384,ACETH@1138384@000255615_3,Cellulase,glycoside hydrolase family 5
4,5,GH39_ALX09806.1,GH39,ALX09806.1,Acetivibrio thermocellus AD2,1138384,ACETH@1138384@000255615_3,Cellulase,Carbohydrate binding family 6
5,6,GH44_ALX08608.1,GH44,ALX08608.1,Acetivibrio thermocellus AD2,1138384,ACETH@1138384@000255615_3,Cellulase,"Cellulose 1,4-beta-cellobiosidase, Cellulase"
...,...,...,...,...,...,...,...,...,...
292,293,CBM50_UKZ49624.1,CBM50,UKZ49624.1,Trichoderma virens Gv29-8,413071,TRIVI@413071@000170995_1,Carbohydrate-binding module,hypothetical protein
295,296,CBM50_UKZ57353.1,CBM50,UKZ57353.1,Trichoderma virens Gv29-8,413071,TRIVI@413071@000170995_1,Carbohydrate-binding module,hypothetical protein
296,297,CBM55_EDR28415.1,CBM55,EDR28415.1,Entamoeba dispar SAW760,370354,ENTHI@294381@000208925_1,Carbohydrate-binding module,Chitotriosidase-1 precursor
297,298,CBM55_EDR24933.1,CBM55,EDR24933.1,Entamoeba dispar SAW760,370354,ENTHI@294381@000208925_1,Carbohydrate-binding module,Chitin binding lectin


Counter({'Cellulase': 52, 'Pectinase': 21, 'Carbohydrate-binding module': 11, 'Hemicellulase': 10, 'Chitinase': 9, 'Chitosanase': 2})


## Write 

In [25]:
repdf.to_csv(f'{milestonedir}/representative_seedgenes_info.tsv')

with open(f'{milestonedir}/representative_seedgenes_list.txt', 'w') as of:
    for seedgene in representatives:
        of.write(seedgene + '\n')